# MAXP 2021初赛数据探索和处理-2

处理Feature，并结合第1部分里得到的不匹配的节点，生成新的Feature，并保存。

In [4]:
import pandas as pd
import numpy as np
import os
import gc

In [5]:
# path
base_path = '/Users/jamezhan/PycharmProjects/MAXP/final_dataset'
publish_path = 'publish'

diff_node_path = os.path.join(base_path, publish_path, 'diff_nodes.csv')
train_nodes_path = os.path.join(base_path, publish_path, 'train_nodes.csv')
val_nodes_path = os.path.join(base_path, publish_path, 'validation_nodes.csv')

### 处理节点的特征

In [6]:
def process_node(line):
    nid, feat_json, label = line.strip().split('\"')
    
    feat_list = [float(feat[1:-1]) for feat in feat_json[1:-1].split(', ')]
    
    if len(feat_list) != 300:
        print('此行数据有问题 {}'.format(line))
    
    return nid[:-1], feat_list, label[1:]

In [7]:
# 下面处理特征Feature并存储亿备后用
# nid_list = []
feature_list = []

with open(train_nodes_path, 'r') as f:
    i = 0
    
    for line in f:
        if i > 0:
            _, features, _ = process_node(line)
#             nid_list.append(nid)
            feature_list.append(features)
        i += 1
        if i % 100000 == 0:
            print('Processed {} train rows'.format(i))
    
with open(val_nodes_path, 'r') as f:
    i = 0
    
    for line in f:
        if i > 0:
            _, features, _ = process_node(line)
#             nid_list.append(nid)
            feature_list.append(features)
        i += 1
        if i % 100000 == 0:
            print('Processed {} validation rows'.format(i))
            
# nid_arr = np.array(nid_list)
feat_arr = np.array(feature_list)

Processed 100000 train rows
Processed 200000 train rows
Processed 300000 train rows
Processed 400000 train rows
Processed 500000 train rows
Processed 600000 train rows
Processed 700000 train rows
Processed 800000 train rows
Processed 900000 train rows
Processed 1000000 train rows
Processed 1100000 train rows
Processed 1200000 train rows
Processed 1300000 train rows
Processed 1400000 train rows
Processed 1500000 train rows
Processed 1600000 train rows
Processed 1700000 train rows
Processed 1800000 train rows
Processed 1900000 train rows
Processed 2000000 train rows
Processed 2100000 train rows
Processed 2200000 train rows
Processed 2300000 train rows
Processed 2400000 train rows
Processed 2500000 train rows
Processed 2600000 train rows
Processed 2700000 train rows
Processed 2800000 train rows
Processed 2900000 train rows
Processed 3000000 train rows
Processed 100000 validation rows
Processed 200000 validation rows
Processed 300000 validation rows
Processed 400000 validation rows
Process

In [8]:
# 删除list以节省内存
del feature_list
gc.collect()

0

In [9]:
# 给未匹配上的419个节点造300维的特征，这里用其他所有节点的300维的平均值来作为他们的特征
# 更好的方法是用每个节点的所有邻居的特征的平均，这里就不搞这么复杂了。
diff_node_feat_arr = np.tile(np.mean(feat_arr, axis=0),(419, 1))

In [11]:
feat_arr = np.concatenate((feat_arr, diff_node_feat_arr), axis=0)
feat_arr.shape

(3655452, 300)

In [ ]:
# 使用Numpy保存特征为.npy格式，以节省存储空间和提高读写速度
with open(os.path.join(base_path, publish_path, './features.npy'), 'wb') as f:
    np.save(f, feat_arr)